# Decoding Strategies Playground  
Interactively compare **deterministic** vs **stochastic** decoding settings—temperature, top‑k, top‑p, presence/frequency penalties—and observe how they shape model outputs.

Works in Google Colab or any local Jupyter (add your `OPENAI_API_KEY`).

In [ ]:
!pip -q install openai ipywidgets

In [ ]:
import os, openai, ipywidgets as w, json
from IPython.display import display, Markdown

os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'sk-')

prompt_box = w.Textarea(
    value='Write a short sci‑fi scene set on Mars.',
    description='Prompt:',
    layout=w.Layout(width='100%', height='80px'))

style_dd = w.Dropdown(
    options=['Deterministic (temp=0, top_p=1)',
             'Greedy (temp=0, top_p=1, best_of=1)',
             'Stochastic Creative (temp=1.0, top_p=0.95)',
             'High Diversity (temp=1.2, top_p=0.8, top_k=40)',
             'Custom'], value='Deterministic (temp=0, top_p=1)',
    description='Preset:')

temp = w.FloatSlider(value=0, min=0, max=2, step=0.1, description='Temperature')
top_p = w.FloatSlider(value=1, min=0.05, max=1, step=0.05, description='Top‑p')
top_k = w.IntSlider(value=0, min=0, max=100, step=5, description='Top‑k')
pres = w.FloatSlider(value=0, min=-2, max=2, step=0.1, description='Presence')
freq = w.FloatSlider(value=0, min=-2, max=2, step=0.1, description='Frequency')

def apply_preset(change):
    if change['new'] == 'Custom': return
    mapping = {
        'Deterministic (temp=0, top_p=1)': (0,1,0,0,0),
        'Greedy (temp=0, top_p=1, best_of=1)': (0,1,0,0,0),
        'Stochastic Creative (temp=1.0, top_p=0.95)': (1.0,0.95,0,0,0),
        'High Diversity (temp=1.2, top_p=0.8, top_k=40)': (1.2,0.8,40,0,0)
    }
    t, p, k, pr, fr = mapping[change['new']]
    temp.value, top_p.value, top_k.value, pres.value, freq.value = t, p, k, pr, fr

style_dd.observe(apply_preset, names='value')

run_btn = w.Button(description='Run 👉')
out = w.Output()

def run(_):
    with out:
        out.clear_output()
        cfg = dict(temperature=temp.value, top_p=top_p.value,
                   presence_penalty=pres.value, frequency_penalty=freq.value,
                   max_tokens=200)
        if top_k.value: cfg['logit_bias'] = {}  # placeholder (top_k not accessible in OpenAI API)
        print('Decoding cfg:', cfg)
        try:
            resp = openai.ChatCompletion.create(
                model='gpt-4o-mini',
                messages=[{'role':'user','content': prompt_box.value}],
                **cfg
            )
            display(Markdown(resp.choices[0].message.content))
        except Exception as e:
            print('Error:', e)

run_btn.on_click(run)

ui = w.VBox([prompt_box, style_dd, temp, top_p, top_k, pres, freq, run_btn, out])
display(ui)
print('Select a preset or tweak parameters, then **Run**.')

---  
### Experiments  
* Compare deterministic vs stochastic for factual Q&A vs creative writing.  
* Increase `presence_penalty` to push novelty.  
* Note: OpenAI chat API doesn’t expose top‑k directly; this slider is illustrative.